# Solution

## Data loading

We've already determined which pixels are "interesting" in the [previous notebook](Interesting.ipynb).

In [1]:
import pandas as pd

interesting = pd.read_pickle('data/interesting.pkl')
interesting.head()

r           c  area  eccentricity  solidity  \
part sequence frame                                                       
test 1        1      1.000000   91.000000     5      0.000000  1.000000   
              1      2.380952  331.000000    21      0.941337  0.724138   
              1      0.250000  338.000000     4      0.790569  1.000000   
              1      0.666667  415.333333     9      0.942349  0.750000   
              1      7.464789  339.471831   142      0.981029  0.835294   

                    is_satellite  
part sequence frame               
test 1        1             None  
              1             None  
              1             None  
              1             None  
              1             None

In [2]:
f'{len(interesting):,}'

'13,398,826'

'8,349,960'

## Feature extraction

In [8]:
import numpy as np
from scipy import stats

def region(img: np.ndarray, r: int, c: int, w: int):
    """Returns the square of length width with (x, y) being at the center."""
    return img[
        max(r - w, 0) : min(r + w + 1, img.shape[0]),
        max(c - w, 0) : min(c + w + 1, img.shape[1])
    ]

def extract_features(img, r, c):
    r3x3 = region(img, r, c, 3).ravel()
    r5x5 = region(img, r, c, 5).ravel()
    r7x7 = region(img, r, c, 7).ravel()
    val = img[r, c]
    return {
        'pixel_value': val,
        '3x3_std': r3x3.std(),
        '3x3_min': val - r3x3.min(),
        '3x3_max': val - r3x3.max(),
        '5x5_std': r5x5.std(),
        '5x5_entropy': stats.entropy(r5x5),
        '5x5_min': val - r5x5.min(),
        '5x5_max': val - r5x5.max(),
        '7x7_std': r7x7.std(),
        '7x7_entropy': stats.entropy(r7x7),
        '7x7_kurtosis': stats.kurtosis(r7x7),
        '7x7_skew': stats.skew(r7x7)
    }

Extract features for each interesting region.

In [9]:
from PIL import Image
import tqdm

samples = {}

# There should be 32000 frames (5 * 1280 + 5 * 5120)
for (part, sequence, frame), locations in tqdm.tqdm(interesting.groupby(['part', 'sequence', 'frame']), position=0):

#for (sequence, frame), locations in tqdm.tqdm(interesting.loc['train'].groupby(['sequence', 'frame']), position=0): 
#    part = 'train'
    
    img = np.asarray(Image.open(f'data/spotGEO/{part}/{sequence}/{frame}.png')).astype(np.float32)
    
    for _, location in locations.iterrows():
    
        r = int(location['r'])
        c = int(location['c'])

        samples.append({
            'part': part,
            'sequence': sequence,
            'frame': frame,
            'r': r,
            'c': c,
            
            'is_satellite': location['is_satellite'],
            'area': location['area'],
            'eccentricity': location['eccentricity'],
            'solidity':  location['solidity'],
            
            **extract_features(img, r=r, c=c)
        })
        
samples = pd.DataFrame(samples)
samples = samples.set_index(['part', 'sequence', 'frame', 'r', 'c'])
samples.to_pickle('data/samples.pkl')
samples.head()

100%|██████████| 32000/32000 [2:36:36<00:00,  3.41it/s]   


is_satellite  area  eccentricity  solidity  \
part sequence frame r c                                                
test 1        1     1 91          None     5      0.000000  1.000000   
                    2 331         None    21      0.941337  0.724138   
                    0 338         None     4      0.790569  1.000000   
                      415         None     9      0.942349  0.750000   
                    7 339         None   142      0.981029  0.835294   

                           pixel_value   3x3_std  3x3_min  3x3_max   5x5_std  \
part sequence frame r c                                                        
test 1        1     1 91          50.0  2.211288     11.0      0.0  2.067833   
                    2 331         50.0  2.782049     11.0     -1.0  2.842636   
                    0 338         50.0  2.564226     11.0      0.0  3.345021   
                      415         42.0  2.315377      4.0     -6.0  2.140389   
                    7 339         66.0  8.833364     26.0     -2.0  7.947879   

                           5x5_entropy  5x5_min  5x5_max   7x7_std  \
part sequence frame r c                                              
test 1        1     1 91      4.342666     12.0      0.0  1.977597   
                    2 331     4.475348     11.0     -5.0  6.136916   
                    0 338     4.186919     11.0     -9.0  6.923370   
                      415     4.188386      4.0     -6.0  1.953611   
                    7 339     4.783086     27.0     -2.0  7.186401   

                           7x7_entropy  7x7_kurtosis  7x7_skew  
part sequence frame r c                                         
test 1        1     1 91      4.904221      0.794483  0.382591  
                    2 331     5.002543      3.388442  1.920722  
                    0 338     4.777256      1.608331  1.577891  
                      415     4.786424      0.004218  0.148943  
                    7 339     5.405223      1.016366  1.443022

In [1]:
import pandas as pd

samples = pd.read_pickle('data/samples.pkl')

## Learning phase

Split into train and test.

In [3]:
from sklearn import utils

X_train = samples.loc['train'].copy()
y_train = X_train.pop('is_satellite').astype(bool)
X_train, y_train = utils.shuffle(X_train, y_train, random_state=42)

try:
    X_test = samples.loc['test'].drop(columns='is_satellite')
except KeyError:
    X_test = None

Do the LGBM CV dance.

In [4]:
from imblearn import pipeline
from imblearn import under_sampling
import lightgbm
from sklearn import metrics
from sklearn import model_selection
from sklearn import utils

model = lightgbm.LGBMClassifier(
    scale_pos_weight=2,
    num_leaves=2 ** 6,
    learning_rate=.01,
    metric='binary',
    random_state=42,
    min_child_samples=30,
    n_estimators=10_000
)

#sampler = under_sampling.RandomUnderSampler()

cv = model_selection.GroupKFold(n_splits=5)
groups = X_train.index.get_level_values('sequence')

oof = pd.Series(dtype=bool, index=X_train.index)
if X_test is not None:
    y_test = pd.DataFrame(index=X_test.index)

for i, (fit_idx, val_idx) in enumerate(cv.split(X_train, y_train, groups=groups)):
    
    X_fit = X_train.iloc[fit_idx]
    y_fit = y_train.iloc[fit_idx]
    X_val = X_train.iloc[val_idx]
    y_val = y_train.iloc[val_idx]
    
    #X_fit, y_fit = sampler.fit_sample(X_fit, y_fit)
    
    model.fit(
        X_fit, y_fit,
        eval_set=[(X_fit, y_fit), (X_val, y_val)],
        eval_names=['fit', 'val'],
        early_stopping_rounds=20,
        verbose=100
    )
    oof.iloc[val_idx] = model.predict(X_val)
    
    if X_test is not None:
        y_test[i] = model.predict_proba(X_test)[:, 1]
    
    print()

print(metrics.classification_report(y_train, oof, digits=4))

Training until validation scores don't improve for 20 rounds
[100]	fit's binary_logloss: 0.00687965	val's binary_logloss: 0.00810302
[200]	fit's binary_logloss: 0.00529186	val's binary_logloss: 0.00682714
[300]	fit's binary_logloss: 0.00456193	val's binary_logloss: 0.00635397
[400]	fit's binary_logloss: 0.0040962	val's binary_logloss: 0.00615418
[500]	fit's binary_logloss: 0.00375419	val's binary_logloss: 0.00602786
[600]	fit's binary_logloss: 0.00350192	val's binary_logloss: 0.00595567
[700]	fit's binary_logloss: 0.00331275	val's binary_logloss: 0.00592096
[800]	fit's binary_logloss: 0.00315079	val's binary_logloss: 0.00588711
[900]	fit's binary_logloss: 0.00301073	val's binary_logloss: 0.00586173
[1000]	fit's binary_logloss: 0.00288577	val's binary_logloss: 0.00583966
[1100]	fit's binary_logloss: 0.00276942	val's binary_logloss: 0.0058195
[1200]	fit's binary_logloss: 0.00265039	val's binary_logloss: 0.00579947
[1300]	fit's binary_logloss: 0.00253746	val's binary_logloss: 0.00578233
[

precision    recall  f1-score   support

       False     0.9987    0.9996    0.9991   1683412
        True     0.9084    0.7537    0.8239      8899

    accuracy                         0.9983   1692311
   macro avg     0.9536    0.8766    0.9115   1692311
weighted avg     0.9982    0.9983    0.9982   1692311

Feature importances.

In [5]:
pd.Series(model.feature_importances_, index=X_train.columns).sort_values(ascending=False)

7x7_skew        15101
3x3_std         15071
7x7_kurtosis    14087
7x7_std         12423
7x7_entropy     11647
5x5_entropy     11460
pixel_value     11069
5x5_std         10079
eccentricity     9074
area             8344
3x3_min          6256
5x5_min          6186
solidity         6077
3x3_max          3789
5x5_max          2977
dtype: int32

## Out-of-fold predictions

In [6]:
%run toolbox.py

In [7]:
oof.head()

sequence  frame  r    c  
1020      1      125  548    False
215       1      80   183    False
844       1      238  574    False
17        2      217  345    False
1019      4      155  422    False
dtype: bool

In [8]:
save_predictions(oof[oof].sort_index(), path='oof.json', n_sequences=1280)

100%|██████████| 1280/1280 [00:02<00:00, 455.82it/s]


In [9]:
!python validation.py oof.json data/spotGEO/train_anno.json

Score: 0.277273, (MSE: 45259.145451)


Score: 0.277273, (MSE: 45259.145451)

## Test predictions

In [10]:
y_test.head()

0         1         2         3         4
sequence frame r c                                                    
1        1     1 91   0.000835  0.000328  0.000325  0.000870  0.000501
               2 331  0.000192  0.000092  0.000350  0.000485  0.000112
               0 338  0.000087  0.000052  0.000058  0.000100  0.000140
                 415  0.000014  0.000012  0.000003  0.000018  0.000008
               7 339  0.000117  0.000059  0.000055  0.000125  0.000093

In [11]:
import zipfile

sightings = y_test.mean(axis='columns') > .4
save_predictions(sightings[sightings], path='submission.json', n_sequences=5120)

with zipfile.ZipFile('submission.zip', mode='w') as f:
    f.write('submission.json')

100%|██████████| 5120/5120 [00:12<00:00, 409.21it/s]


Next is [PostProcessing.ipynb](PostProcessing.ipynb).